In [1]:
%cd /content
!git clone https://github.com/vohoaidanh/dino.git

/content
Cloning into 'dino'...
remote: Enumerating objects: 183, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 183 (delta 0), reused 4 (delta 0), pack-reused 175 (from 1)
Receiving objects: 100% (183/183), 24.49 MiB | 18.17 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [2]:
USE_COLAB = True
WORKING_DIR = 'workspace'
if USE_COLAB:
    WORKING_DIR = 'content'


In [3]:
%cd /$WORKING_DIR/dino

/content/dino


In [5]:
#Download Train, val set
#https://drive.google.com/file/d/1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z/view?usp=drive_link
import gdown
#file_id = '1JBDx2BPVSoADxhLFMJpd8zSFQyD_SVvI' #Progan train/val 4 class [car, cat, chair, horse] 13Gb
file_id = '1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z' #Progan train/val 4 class [car, cat, chair, horse]
destination = f'/{WORKING_DIR}/Progan_train.zip'  # Desired file name and extension
# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'
# Download the file
gdown.download(url, destination, quiet=False)
#Unzip
!mkdir /$WORKING_DIR/datasets
!unzip -q $destination -d /$WORKING_DIR/datasets/ForenSynths_train

Downloading...
From (original): https://drive.google.com/uc?id=1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z
From (redirected): https://drive.google.com/uc?id=1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z&confirm=t&uuid=e3cccb4f-9644-467a-9be0-5c59b3811e00
To: /content/Progan_train.zip

  0%|          | 0.00/2.64G [00:00<?, ?B/s]
  0%|          | 1.57M/2.64G [00:00<02:53, 15.2MB/s]
  0%|          | 4.72M/2.64G [00:00<02:35, 16.9MB/s]
  0%|          | 8.91M/2.64G [00:00<02:15, 19.4MB/s]
  1%|          | 17.3M/2.64G [00:00<01:36, 27.3MB/s]
  1%|          | 25.7M/2.64G [00:01<01:38, 26.5MB/s]
  1%|▏         | 34.1M/2.64G [00:01<01:14, 34.9MB/s]
  2%|▏         | 42.5M/2.64G [00:01<01:11, 36.2MB/s]
  2%|▏         | 50.9M/2.64G [00:01<01:06, 38.9MB/s]
  2%|▏         | 59.2M/2.64G [00:01<01:02, 41.4MB/s]
  3%|▎         | 67.6M/2.64G [00:01<00:57, 44.8MB/s]
  3%|▎         | 76.0M/2.64G [00:02<00:54, 47.4MB/s]
  3%|▎         | 84.4M/2.64G [00:02<01:01, 41.8MB/s]
  4%|▎         | 92.8M/2.64G [00:02<00:55, 45.8MB/s]
  4

In [16]:
!wget https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain_full_checkpoint.pth .

--2024-12-26 14:52:34--  https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain_full_checkpoint.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.78, 13.226.210.111, 13.226.210.25, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 352247243 (336M) [application/zip]
Saving to: ‘dino_deitsmall16_pretrain_full_checkpoint.pth’

dino_deitsmall16_pr 100%[===================>] 335.93M  63.8MB/s    in 5.6s    

2024-12-26 14:52:40 (60.4 MB/s) - ‘dino_deitsmall16_pretrain_full_checkpoint.pth’ saved [352247243/352247243]

--2024-12-26 14:52:40--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2024-12-26 14:52:40--
Total wall clock time: 5.8s
Downloaded: 1 files, 336M in 5.6s (60.4 MB/s)


In [ ]:
import os
import shutil

def move_images_from_real_folders(source_folder, destination_folder, folder_name='0_real'):
    # Kiểm tra nếu thư mục đích tồn tại, nếu không tạo mới
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Duyệt qua tất cả các thư mục trong thư mục nguồn
    for root, dirs, files in os.walk(source_folder):
        # Kiểm tra nếu thư mục con có tên '0_real'
        if '1_fake' in dirs:
            # Lấy đường dẫn tới thư mục '0_real'
            real_folder_path = os.path.join(root, '1_fake')

            # Duyệt qua tất cả các tệp trong thư mục '0_real'
            for file in os.listdir(real_folder_path):
                file_path = os.path.join(real_folder_path, file)

                # Kiểm tra nếu tệp là hình ảnh (có thể mở rộng nếu cần)
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                    # Tạo đường dẫn mới để di chuyển tệp tới thư mục đích
                    destination_path = os.path.join(destination_folder, file)

                    # Di chuyển tệp
                    shutil.move(file_path, destination_path)
                    print(f"Đã di chuyển {file} từ {file_path} tới {destination_path}")

# Ví dụ sử dụng hàm
source_folder = f'/{WORKING_DIR}/datasets/ForenSynths_train/train'  # Thư mục nguồn

for folder_name in ['0_real', '1_fake']:
  folder_name = '1_fake'
  destination_folder = f'/{WORKING_DIR}/dataset/{folder_name}'  # Thư mục đích
  move_images_from_real_folders(source_folder, destination_folder,folder_name)


In [ ]:
import torch
!python main_dino.py --arch vit_small_from_checkpoint --data_path /$WORKING_DIR/dataset --output_dir /$WORKING_DIR/dino/saving_dir
#python -m torch.distributed.launch --nproc_per_node=8 main_dino.py --arch vit_small --data_path /$WORKING_DIR/dataset/train --output_dir /$WORKING_DIR/dino/saving_dir

Using cache found in /root/.cache/torch/hub/facebookresearch_xcit_main
/usr/local/lib/python3.10/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Will run the code on one GPU.
| distributed init (rank 0): env://
[rank0]:[W1226 14:58:02.027040205 ProcessGroupNCCL.cpp:4115] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect.Specify device_ids in barrier() to force use of a particular device,o

In [18]:
!rm -r /content/dino/saving_dir